# Prueba o test A/B 

Deacuerdo  a la [wikipedia](https://es.wikipedia.org/wiki/Prueba_A/B) : 

_El término test A/B se utiliza en el ámbito del Marketing Digital y la Analítica web para describir experimentos aleatorios con dos variantes, A y B, siendo una la de control y la otra la variante. Otra forma de referirse generalmente a los test A/B es con el término split test, aunque este último método se aplica cuando se realizan experimentos con más de dos variantes._


En este notebook desarrollaremos (como a lo largo de las clases) el proceso de análisis de un experimento/prueba/test A/B, desde formular una hipótesis, probarla y finalmente interpretar los resultados. Para que esto sea facil de replicar , usaremos un conjunto de datos de Kaggle que contiene los resultados de una prueba A/B en lo que parecen ser 2 diseños diferentes de una página de sitio web (_old_page_ vs. _new_page_).

Esto es lo que haremos: 
  * Diseñando nuestro experimento 
  * Recopilar y preparar los datos 
  * Visualizando los resultados 
  * Probando la hipótesis 
  * Sacar conclusiones

Para hacerlo un poco más realista, aquí hay un escenario potencial para nuestro estudio:

> Imaginemos que trabaja en el equipo de productos de una empresa de comercio electrónico online de tamaño medio. El diseñador de UX trabajó muy duro en una nueva versión de la página del producto, con la esperanza de que conduzca a una mayor tasa de conversión. El gerente de producto (PM) le dijo que la tasa de conversión actual es de aproximadamente un 13% en promedio durante todo el año, y que el equipo estaría contento con un aumento del 2%, lo que significa que el nuevo diseño se considerará un éxito si aumenta la tasa de conversión al 15%.

Antes de implementar el cambio, el equipo se sentiría más cómodo probándolo en una pequeña cantidad de usuarios para ver cómo funciona, por lo que sugiere ejecutar una prueba A / B en un subconjunto de usuarios de su base de usuarios.

## 1.Diseñando nuestro experimento
Empezemos diseñando nuestro experimento :

### Formular una hipótesis

Lo primero es lo primero, queremos asegurarnos de formular una hipótesis al comienzo de nuestro proyecto. Esto asegurará que nuestra interpretación de los resultados sea correcta y rigurosa.

Dado que no sabemos si el nuevo diseño funcionará mejor o peor (¿o igual?) Que nuestro diseño actual, elegiremos una prueba de dos colas (*two-tailed test*):

$$H_0 :p = p_0$$

$$H_a : p \neq p_0$$

donde $p$ y $p_0$ representan la tasa de conversión del diseño nuevo y antiguo, respectivamente. También estableceremos un nivel de confianza del 95%:

$$\alpha = 0.05$$

El valor $\alpha$ es un umbral que establecemos, por el cual decimos “si la probabilidad de observar un resultado como extremo o mayor (valor p) es menor que $\alpha$, entonces rechazamos la hipótesis nula”. Dado que nuestro $\alpha$ = 0.05 (que indica un 5% de probabilidad), nuestra confianza (1 - $\alpha$) es del 95%.

No se preocupe si no está familiarizado con lo anterior, todo esto realmente significa que cualquiera que sea la tasa de conversión que observemos para nuestro nuevo diseño en nuestra prueba, queremos tener un 95% de confianza en que es estadísticamente diferente de la tasa de conversión de nuestro antiguo diseño, antes de que decidamos rechazar la hipótesis nula $H_0$.

### Elegir las variables

Para nuestra prueba, necesitaremos dos grupos:

  * Un grupo de control: se les mostrará el diseño anterior

  * Un grupo de prueba (tratamiento o experimental): se les mostrará el nuevo diseño

Esta será nuestra Variable Independiente. La razón por la que tenemos dos grupos, a pesar de que conocemos la tasa de conversión inicial, es que queremos controlar otras variables que podrían tener un efecto en nuestros resultados, como la estacionalidad: al tener un grupo de control, podemos comparar directamente sus resultados con el tratamiento. grupo, porque la única diferencia sistemática entre los grupos es el diseño de la página del producto, y por lo tanto podemos atribuir cualquier diferencia en los resultados a los diseños.

Para nuestra Variable dependiente (es decir, lo que estamos tratando de medir), estamos interesados ​​en capturar la tasa de conversión. Una forma en que podemos codificar esto es mediante cada sesión de usuario con una variable binaria:

  * 0: el usuario no compró el producto durante esta sesión de usuario

  * 1 - El usuario compró el producto durante esta sesión de usuario

De esta manera, podemos calcular fácilmente la media de cada grupo para obtener la tasa de conversión de cada diseño.

### Elegir un tamaño de muestra
Es importante tener en cuenta que, dado que no probaremos toda la base de usuarios (nuestra población), las tasas de conversión que obtendremos serán, inevitablemente, solo estimaciones de las tasas reales.

El número de personas (o sesiones de usuario) que decidamos capturar en cada grupo tendrá un efecto en la precisión de nuestras tasas de conversión estimadas: cuanto mayor sea el tamaño de la muestra, más precisas serán nuestras estimaciones (es decir, cuanto menores sean nuestros intervalos de confianza), mayor es la posibilidad de detectar una diferencia en los dos grupos, si está presente.

Por otro lado, cuanto más grande es nuestra muestra, más caro (y poco práctico) se vuelve nuestro estudio.

Entonces, **¿cuántas personas deberíamos tener en cada grupo?**

El tamaño de la muestra que necesitamos se estima mediante algo llamado [análisis de potencia](https://research.usu.edu//irb/wp-content/uploads/sites/12/2015/08/A_Researchers_Guide_to_Power_Analysis_USU.pdf), y depende de algunos factores:

  * **Potencia de la prueba (1 - β)**: representa la probabilidad de encontrar una diferencia estadística entre los grupos de nuestra prueba cuando existe una diferencia. Esto generalmente se establece en 0.8 por convención (mas informacion en la [wiki](https://en.wikipedia.org/wiki/Power_of_a_test))

  * **Valor alfa (α)**: el valor crítico que establecimos anteriormente en 0.05

  * **Tamaño del efecto**: qué tan grande es la diferencia que esperamos que haya entre las tasas de conversión

Dado que nuestro equipo estaría contento con una diferencia del 2%, podemos usar el 13% y el 15% para calcular el tamaño del efecto que esperamos.

Afortunadamente,y como ya lo hemos hecho en notebooks pasados,  Python se encarga de todos estos cálculos por nosotros:

In [ ]:
# Packages imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

# Some plot styling preferences
plt.style.use('seaborn-whitegrid')
font = {'family' : 'Helvetica',
        'weight' : 'bold',
        'size'   : 14}

mpl.rc('font', **font)
effect_size = sms.proportion_effectsize(0.13, 0.15)    # Calculating effect size based on our expected rates

required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    )                                                  # Calculating sample size needed

required_n = ceil(required_n)                          # Rounding up to next whole number                          

print(required_n)

Necesitaríamos al menos **4720 observaciones para cada grupo**.

Habiendo establecido el parámetro de potencia en 0.8 en la práctica significa que si existe una diferencia real en la tasa de conversión entre nuestros diseños, asumiendo que la diferencia es la que estimamos (13% vs 15%), tenemos alrededor del 80% de posibilidades de detectarla. como estadísticamente significativo en nuestra prueba con el tamaño de la muestra que calculamos.

## 2.Recopilar y preparar los datos
Entonces, ahora que tenemos nuestro tamaño de muestra requerido, necesitamos recopilar los datos. Por lo general, en este punto, trabajaría con su equipo para configurar el experimento, probablemente con la ayuda del equipo de ingeniería, y se aseguraría de recopilar suficientes datos según el tamaño de muestra necesario.

Sin embargo, usaremos un conjunto de datos que encontramos en línea, para simular esta situación:

  > Descarga el conjunto de datos de Kaggle

  >  Almacenar la data en una variable de tipo [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)

  > Verifique y limpie los datos según sea necesario

  > Muestra aleatoria de $n = 4720$ filas del DataFrame para cada grupo

* _Nota_: Normalmente, no necesitaríamos realizar el paso 4, esto es solo por el ejercicio

Nuestro siguiente paso es cargar los datos en nuestro notebook para empezar con el analisis :


In [ ]:
# 1era forma : descargamos los datos de kaggle y los subimos a nuestro notebook :
# descargamos la data :
# https://www.kaggle.com/zhangluyuan/ab-testing?select=ab_data.csv
# Luego cargamos estos datos en google colab

from google.colab import files

uploaded = files.upload() 

# La otra opcion es subirlo al repo de datos de robintux y desde ahi cargar la data.

In [ ]:
# 1era forma
df = pd.read_csv('ab_data.csv')

# 2da forma
# df= pd.read_csv("https://raw.githubusercontent.com/robintux/Datasets4StackOverFlowQuestions/master/ab_data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
# Para asegurarse de que todo el grupo de control esté viendo la página anterior y viceversa
pd.crosstab(df['group'], df['landing_page'])



Hay 294478 filas en el DataFrame, cada una de las cuales representa una sesión de usuario, así como 5 columnas:

> $user\_id$ - El ID del usuario de cada sesion

> $timestamp$ - Informacion temporal de la sesion.

> $group$ - A qué grupo se asignó el usuario para esa sesión
(_control_, _prueba_)

> $landing\_page$ - Qué diseño vio cada usuario en esa sesión
(_old_page_, _new_page_)

> converted - Si la sesión terminó en una conversión o no
(variable binaria, $0$ = _no se convirtio_ , $1$= Se convirtio )

De hecho, solo usaremos las variables $group$ y $converted$ para nuestro analisis análisis.

Antes de continuar y muestrear los datos para obtener nuestro subconjunto, asegurémonos de que no haya usuarios que hayan sido muestreados varias veces.

In [ ]:
session_counts = df['user_id'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Hay {multi_users} usuarios que aparecen varias veces en el conjunto de datos')

De hecho, hay 3894 usuarios que aparecen más de una vez. Dado que el número es bastante bajo, continuaremos y los eliminaremos del DataFrame para evitar muestrear a los mismos usuarios dos veces.


In [ ]:
users_to_drop = session_counts[session_counts > 1].index

df = df[~df['user_id'].isin(users_to_drop)]
print(f'Despues de limpiar esos usuarios que aparecen multiples veces \nEl dataframe ahora tiene {df.shape[0]} filas')

### Muestreo

Ahora que nuestro DataFrame es agradable y limpio, podemos continuar y muestrear n = 4720 entradas para cada uno de los grupos. Podemos usar el método DataFrame.sample () de pandas para hacer esto, que realizará un muestreo aleatorio simple para nosotros.

Nota: Configuraremos random_state = 22 para que los resultados sean reproducibles si desea seguir en su propio Notebook: simplemente use random_state = 22 en su función y debería obtener la misma muestra.



In [ ]:
control_sample = df[df['group'] == 'control'].sample(n=required_n, random_state=22)
treatment_sample = df[df['group'] == 'treatment'].sample(n=required_n, random_state=22)

ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)
ab_test

In [ ]:
ab_test.info()

In [ ]:
ab_test['group'].value_counts()

Genial, parece que todo salió según lo planeado y ahora estamos listos para analizar nuestros resultados.


## 3.Visualizando los resultados
Lo primero que podemos hacer es calcular algunas estadísticas básicas para tener una idea de cómo se ven nuestras muestras.



In [ ]:
conversion_rates = ab_test.groupby('group')['converted']

std_p = lambda x: np.std(x, ddof=0)              # Std. deviation of the proportion
se_p = lambda x: stats.sem(x, ddof=0)            # Std. error of the proportion (std / sqrt(n))

conversion_rates = conversion_rates.agg([np.mean, std_p, se_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.3f}')

A juzgar por las estadísticas anteriores, parece que nuestros dos diseños funcionaron de manera muy similar, con nuestro nuevo diseño un poco mejor, aprox. 12,3% frente a 12,6% de tasa de conversión.

Graficar los datos facilitará la comprensión de estos resultados:



In [ ]:
plt.figure(figsize=(8,6))

sns.barplot(x=ab_test['group'], y=ab_test['converted'], ci=False)

plt.ylim(0, 0.17)
plt.title('Ratio de conversion porgrupo', pad=20)
plt.xlabel('Grupos', labelpad=15)
plt.ylabel('Converted (proporcion)', labelpad=15);
plt.show()

Las tasas de conversión de nuestros grupos están muy cerca. También tenga en cuenta que la tasa de conversión del grupo de control es menor de lo que hubiéramos esperado dado lo que sabíamos sobre nuestro promedio. tasa de conversión (12,3% frente a 13%). Esto demuestra que existe cierta variación en los resultados cuando se toman muestras de una población.

Entonces… __el valor del grupo de tratamiento es mayor. ¿Es esta diferencia estadísticamente significativa?__

## 4.Prueba de la hipótesis

El último paso de nuestro análisis es probar nuestra hipótesis. Dado que tenemos una muestra muy grande, podemos usar la aproximación normal para calcular nuestro valor p (es decir, prueba z).

Una vez más, Python facilita todos los cálculos. Podemos usar el módulo statsmodels.stats.proportion para obtener el valor p y los intervalos de confianza:



In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
control_results = ab_test[ab_test['group'] == 'control']['converted']
treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

## 5.Sacar conclusiones

Dado que nuestro valor p = 0,732 está muy por encima de nuestro umbral α = 0,05, no podemos rechazar la hipótesis nula Hₒ, lo que significa que nuestro nuevo diseño no tuvo un rendimiento significativamente diferente (y mucho menos mejor) que el anterior.

Además, si observamos el intervalo de confianza para el grupo de tratamiento ([0.116, 0.135] o 11.6-13.5%) notamos que:

  > Incluye nuestro valor de referencia del 13% de tasa de conversión.

  > No incluye nuestro valor objetivo del 15% (el aumento del 2% al que aspiramos)

Lo que esto significa es que es más probable que la tasa de conversión real del nuevo diseño sea similar a nuestra línea de base, en lugar del objetivo del 15% que esperábamos. Esta es una prueba más de que no es probable que nuestro nuevo diseño sea una mejora con respecto a nuestro diseño anterior, y que, lamentablemente, ¡volvemos a la mesa de dibujo!
